@kenpomeroy: When is the last time Arizona/ASU/UCLA/USC went 0-8 at home in the same week? I leave it as an exercise to the reader on account of laziness and the answer probably being never.
https://twitter.com/kenpomeroy/status/1094434062712750081

In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.simplefilter('ignore')

#### Import data and select only Arizona/ASU/UCLA/USC home games:

In [2]:
# raw data from sports-reference (see separate script):
games_df = pd.read_csv('data/p12_conference_games.csv')
games_df.columns

Index(['Date', 'Away', 'Away_Pts', 'Home', 'Home_Pts', 'OT', 'Notes',
       'Conference', 'Year'],
      dtype='object')

In [3]:
team_lst = ['Arizona','Arizona State','UCLA','Southern California']
incl_df = games_df[(games_df['Home'].isin(team_lst)) & (games_df['Year']<2019)]

In [4]:
# check all years are included:

incl_df['Year'].describe()

rng = range(1950,2019)
print(list(set(rng) - set(incl_df['Year'].values)))

[]


#### Summarize results by week:

In [5]:
# reformat date:
incl_df['date_txt'] = incl_df['Date'].apply(lambda x: str(x)[5:])
incl_df['date_val'] = pd.to_datetime(incl_df['date_txt'], format="%b %d, %Y")

# group by week
incl_df['Week'] = incl_df['date_val'].apply(lambda x: x.strftime('%W'))
# change to calendar year
incl_df['Year'] = np.where(incl_df['Week']>"40", incl_df['Year']-1, incl_df['Year'])
incl_df['Week_Year'] = incl_df['Year'].map(str) + "_" + incl_df['Week'].map(str)

In [6]:
# identify losers
incl_df['Home_Mgn'] = incl_df['Home_Pts'] - incl_df['Away_Pts']
incl_df['Home_Win'] = np.where(incl_df['Home_Mgn']>0, 1, 0)

In [7]:
# group by week and identify number of home games etc.
out_df = incl_df.groupby(['Week_Year'])['Home_Win'].agg(['count','sum'])\
                .rename(columns={'count':'Home_games', 'sum':'Wins'})

out_df['Losses'] = out_df['Home_games'] - out_df['Wins']


In [8]:
# filter for 8-home-game weeks:
out_df = out_df[out_df['Home_games']==8]

## Output

In [9]:
samples = len(out_df)
min_wins = out_df['Wins'].min()
max_losses = 8 - min_wins
print("Number of 8-home-game weeks since 1980: %s" % samples)
print("Worst record for Arizona/ASU/UCLA/USC: %s-%s" % (min_wins, max_losses))

Number of 8-home-game weeks since 1980: 35
Worst record for Arizona/ASU/UCLA/USC: 3-5


##### Full history:

In [10]:
out_df

,Home_games,Wins,Losses
Week_Year,,,
1981_08,8,5,3
1982_09,8,7,1
1988_01,8,5,3
1988_09,8,5,3
1990_09,8,7,1
1991_09,8,8,0
1996_05,8,5,3
1996_10,8,4,4
1997_00,8,7,1
